In [30]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from tqdm import tqdm
import seaborn as sns
from random import randint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


In [45]:
secrets_file = open("secrets.txt","r")
string = secrets_file.read()
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()


In [55]:
secrets_dict

In [53]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [84]:
# sp.categories(limit = 50)

In [36]:
model = load('kmeans10.joblib')
scaler = load('scaler.pkl')

In [73]:
feature_names = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
            'duration_ms'
           ]

In [74]:
clustered_tracks = pd.read_csv('clustered_tracks.csv')
# clustered_tracks

In [75]:
top_songs = pd.read_csv('top_tracks.csv')
top_songs

,Song,Artist
0,Rich Men North of Richmond,Oliver Anthony Music
1,Margaritaville,Jimmy Buffett
2,I Remember Everything (feat. Kacey Musgraves),Zach Bryan
3,Come Monday,Jimmy Buffett
4,Paint The Town Red,Doja Cat
...,...,...
89,Holy Forever,Chris Tomlin
90,Thank God,Kane Brown & Katelyn Brown
91,Walkin' On the Sun,Smash Mouth
92,Unstoppable,Sia


In [76]:
song_title = input("Enter song title: ")

Enter song title: Margaritaville


In [27]:
song_artist = input("Enter song artist: ")

Enter song artist: Jimmy Buffett


In [51]:
def recommend_song(title, artist):
    if title and artist:
        song_in_top_songs = top_songs[(top_songs['Song'] == title) & (top_songs['Artist'] == artist)]
    elif title:
        song_in_top_songs = top_songs[top_songs['Song'] == title]
    elif artist:
        song_in_top_songs = top_songs[top_songs['Artist'] == artist]
    else:
        song_in_top_songs = top_songs.iloc[0:0] 

    
    # Recommend from Top Songs
    if not song_in_top_songs.empty:
        recommendations = top_songs.drop(song_in_top_songs.index).sample(10)  # Recommending 3 random songs from top songs
        return recommendations

    # If song is not in top songs, return a placeholder for now (will handle in next steps)
    print("Song not in top songs. Fetching recommendations from clusters...")
    return retrieve_scale_predict(title, artist)


In [40]:
recommend_song(song_title, song_artist)

,Song,Artist
83,Thought You Should Know,Morgan Wallen
69,I Love My Body,HWASA
33,Dreams,Fleetwood Mac
26,All Star,Smash Mouth
54,Photograph,Nickelback
90,Thank God,Kane Brown & Katelyn Brown
13,White Horse,Chris Stapleton
50,Driving Back There in My Mind,Miranda Lambert
59,Gypsy,Fleetwood Mac
9,Dream Weaver,Gary Wright


In [89]:
def get_song_features(song_title, song_artist):
    client_credentials_manager = SpotifyClientCredentials(client_id="YOUR_CLIENT_ID", client_secret="YOUR_CLIENT_SECRET")
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    
    # Search for the track
    results = sp.search(q=f'track:{song_title} artist:{song_artist}', type='track', limit=1)
    if not results['tracks']['items']:
        return None  # No track found
    
    track_id = results['tracks']['items'][0]['id']

    # Retrieve audio features
    features = sp.audio_features(track_id)[0]

    # Convert to a format suitable for the scaler and model (excluding non-numeric features)
    feature_values = [features[key] for key in feature_names]
#     print(feature_values)
    return [feature_values]

In [103]:
def retrieve_scale_predict(song_title, song_artist):
    scaled_features = scaler.transform(get_song_features(song_title, song_artist))
    predicted_cluster = model.predict(scaled_features)[0]
    sample10 = clustered_tracks[clustered_tracks['cluster']==predicted_cluster].sample(10)
    print(f'predicted_cluster: {predicted_cluster}')
    return sample10[['title','artists','uri']]

In [49]:
song_title_2 = input("Enter song title: ")

Enter song title: Holiday Nostalgia


In [50]:
song_artist_2 = input("Enter song artist: ")

Enter song artist: Brunette


In [99]:
recommend_song(song_title_2, song_artist_2)

Song not in top songs. Fetching recommendations from clusters...


/Users/lilitdavidyan/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,title,artists,uri
69783,Mama Said,Metallica,spotify:track:4ePx4Js87KSIVMLL4RD7vX
78180,Vampiresa,Los Tigrillos,spotify:track:26uwjXS3gdxAIceTXWrXAO
21871,Seven Screws,Einstürzende Neubauten,spotify:track:4kPEY7J2BqiLZIZ3JIIeMz
11811,It's Love,Jill Scott,spotify:track:2U7X7mPD4wSp0capCGfpmM
13828,너 때문에 미쳐,T-ARA,spotify:track:5vCXnSlbAEkuJ3w2O0ALan
74600,Love at First Sight,Kylie Minogue,spotify:track:6ua8mbyHXTHpAmGyGpFAS2
52013,Sandman,"America, George Martin",spotify:track:5DSQr9VZlvL2wzIAU95w2m
40746,Take Five,Dave Brubeck,spotify:track:5UbLKRX1qVROlwDFOooEvG
94863,SHAKE SUMN (with Sexyy Red) - REMIX,"DaBaby, Sexyy Red",spotify:track:3IxGpAFxBAAfCQ6dv6sBiw
17529,PA PA PA,"Matt Paris, Cornetto, IAmChino",spotify:track:5wuub0qkR7iGjOLkZe4Uod


In [100]:
song_title_3 = input("Enter song title: ")

Enter song title: Vampiresa


In [101]:
song_artist_3 = input("Enter song artist: ")

Enter song artist: Los Tigrillos


In [104]:
recommend_song(song_title_3, song_artist_3)

Song not in top songs. Fetching recommendations from clusters...
predicted_cluster: 0


/Users/lilitdavidyan/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,title,artists,uri
43843,Bebete Vãobora,Jorge Ben Jor,spotify:track:3vST7tho4iT90949PEtM7i
8402,Algodón,Lasso,spotify:track:4h7c7tqYjCBdWz8SPZh3bw
20884,Save Me - Disco Mix,"Marvin & Guy, Hard Ton",spotify:track:6IDonbKy642ltu7P5qimlM
23026,KAMASHA - كماشه (feat. Wezza Montaser),"Mahdy Madness, Wezza Montaser",spotify:track:0TRvIDRqRemga0vsrSaiwf
81524,"Ratfinks, Suicide Tanks And Cannibal Girls",White Zombie,spotify:track:6PDBcqHNIg5cOojcCkhxYM
49884,Take Me in Your Arms (Rock Me a Little While),The Doobie Brothers,spotify:track:5SUECwnqVcLWN32o1BNpQn
74372,U Don't Know,JAY-Z,spotify:track:4O7oKy8YADaBrbehcPl0kE
7498,Moonlight Rendezvous,Beast In Black,spotify:track:6P5Ma1HFYDsgOpLaGu6Jkz
18257,Daydreaming,"Milky Chance, Tash Sultana",spotify:track:3v5OftOHxaVpaaJ8NArHos
2728,The Way I Do,July 7,spotify:track:4apnPdCj3uDuqMjGpqQcuc


In [122]:
song_title_4 = input("Enter song title: ")

Enter song title: Something in the Orange


In [123]:
song_artist_4 = input("Enter song artist: ")

Enter song artist: Zach Bryan


In [124]:
recommend_song(song_title_4, song_artist_4)

Song not in top songs. Fetching recommendations from clusters...
predicted_cluster: 9


/Users/lilitdavidyan/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,title,artists,uri
33558,Riders In the Sky,Stan Jones,spotify:track:3cwVUCWSav17exXcJGF1uB
88965,You Are My Sunshine,Nursery Rhymes 123,spotify:track:0tIwgdXaGSp9rYvjVNAoP7
53368,September Morn,Neil Diamond,spotify:track:2CXkN6UAb3jAFwz4VbsDq0
21703,Das ist alles von der Kunstfreiheit gedeckt,Danger Dan,spotify:track:4ioQKdmkdBTkqjhfHeu4YU
26021,Night and Day,Joséphine Baker,spotify:track:0j6jt4SSfcKurGXy2sGF1F
23144,Wildfire,Watchhouse,spotify:track:0EIS2zaB3aQGtKOvzrodOU
90702,Delicate - Recorded at The Tracking Room Nashv...,Taylor Swift,spotify:track:1Fj0T1XKuhRu7EZzJ7NJIK
36767,It's Over,Roy Orbison,spotify:track:3HZMEIZY9Z6GdhPaG5bAK2
33067,Marian The Librarian,Robert Preston,spotify:track:0SsoCpGsSz32oyQjsvDZFu
40090,Keep Your Arms Around Me,Otis Redding,spotify:track:0JoqdwiF0FKwK1XubPjb85
